In [1]:
from datetime import datetime, timedelta
import logging
import os

from churchtools_api.churchtools_api import ChurchToolsApi

from communi_api.communi_api import CommuniApi
from communi_api.churchToolsActions import delete_event_chats, create_event_chats, get_x_day_event_ids

# Init connection and logging

In [2]:
logging.basicConfig(filename='logs/main.py.log', encoding='utf-8',
    format="%(asctime)s %(name)-10s %(levelname)-8s %(message)s",
    level=logging.DEBUG)

if 'COMMUNI_TOKEN' in os.environ:
    COMMUNI_TOKEN = os.environ['COMMUNI_TOKEN']
    COMMUNI_SERVER = os.environ['COMMUNI_SERVER']
    COMMUNI_APPID = os.environ['COMMUNI_APPID']
    CT_TOKEN =  os.environ['CT_TOKEN']
    CT_DOMAIN =  os.environ['CT_DOMAIN']
    logging.info('using connection details provided with ENV variables')
else:
    from secure.config import token, rest_server, communiAppId
    COMMUNI_TOKEN = token
    COMMUNI_SERVER = rest_server
    COMMUNI_APPID = communiAppId
    from secure.config import ct_token, ct_domain
    CT_TOKEN = ct_token
    CT_DOMAIN =  ct_domain
    logging.info('using connection details provided from secrets folder')

communi_api = CommuniApi(COMMUNI_SERVER, COMMUNI_TOKEN, COMMUNI_APPID)
ct_api = ChurchToolsApi(CT_DOMAIN, CT_TOKEN)
logging.debug(communi_api)
logging.debug(ct_api)


# Next 14 day events
* This will delete all events of the past 14 days
* and create new / update all "relevant" events for the next 15 days

In [3]:
reference_day = datetime.today()
reference_day + timedelta(days=1)
event_ids = get_x_day_event_ids(ct_api, reference_day, -14)
for event in event_ids:
    print(f"{ct_api.get_events(eventId=event)}")
delete_event_chats(ct_api, communi_api, event_ids)

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

Create new days for 15 days

In [ ]:
reference_day = datetime.today()
reference_day + timedelta(days=1)
event_ids = get_x_day_event_ids(ct_api, reference_day, 15)
for event in event_ids:
    print(f"{ct_api.get_events(eventId=event)}")
create_event_chats(ct_api, communi_api, event_ids, only_relevant=True)



## Specific Event
create single event by ID

In [ ]:
event_ids = [2240]
#create_event_chats(ct_api, communi_api, event_ids, only_relevant=True)